1.	Level  0
2.	Count of  0(False)  =  1
3.	Count of  1(True)  =  3
4.	Current Entropy  is =  0.811278124459
5.	Splitting on feature  X1  with gain ratio  0.311278124459

*************************************************************

1.	Level  1
2.	Count of  0  =  1
3.	Count of  1  =  1
4.	Current Entropy is =  1.0
5.	Splitting on feature  X2  with gain ratio  1.0

*************************************************************

1.	Level  2
2.	Count of  0  =  1
3.	Current Entropy  is =  0.0
4.	Reached leaf Node

*************************************************************

1.	Level  2
2.	Count of  1  =  1
3.	Current Entropy  is =  0.0
4.	Reached leaf Node

*************************************************************

1.	Level  1
2.	Count of  1  =  2
3.	Current Entropy  is =  0.0
4.	Reached leaf Node



In [2]:
import numpy as np

In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [4]:
def level(X):
    mean = X.mean()
    one_third_value = mean/2
    two_third_value = 1.5 * mean
    for i in range(len(X)):
        if X[i] < one_third_value:
            X[i] = 1
        elif X[i] < mean:
            X[i] = 2
        elif X[i] < two_third_value:
            X[i] = 3
        else:
            X[i] = 4
    return X

In [5]:
iris_dataset = datasets.load_iris()
X = iris_dataset.data
Y = iris_dataset.target

for i in range(X.shape[1]):
    X[:, i] = level(X[:, i])
    
features = iris_dataset.feature_names
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [6]:
def entropy(Y, possible_classes):
    entropy = 0
    total_count = len(Y)
    for y in possible_classes :
        count_y = (Y == y).sum()
        p_i = count_y/total_count
        entropy -=  p_i * np.log(p_i)
    return entropy

def split(X, Y, feature):
    splits = np.zeros(len(Y))
    possible_values_current_feature = list(set( X[:, feature] ))

    group_label = 0
    for value in possible_values_current_feature :
        splits[ X[:, feature] == value ] = group_label
        group_label += 1
    return splits

def info_gain_on_f(X, Y, feature):
    info_gain = 0
    split_info = 0
    mod_Y = len(Y)
    
    splits = split(X, Y, feature)
    
    #print(splits)
    
    unique_splits = list(set(splits))
    
    for s in unique_splits:
        
        indices = (splits == s)
        
        X_split_i = X[indices, :]
        Y_split_i = Y[indices]
        
        #print([X_split_i, Y_split_i])
        
        mod_Y_i = len(Y_split_i)

        ratio_mod_Y_i_to_mod_Y = (mod_Y_i/mod_Y)
        possible_classes_Y_i = list(set(Y_split_i))
        info_gain += ((ratio_mod_Y_i_to_mod_Y) * entropy( Y_split_i, possible_classes_Y_i))
        split_info -= (ratio_mod_Y_i_to_mod_Y * np.log(ratio_mod_Y_i_to_mod_Y))
        
    return info_gain, split_info
    
def decision_tree(X, Y, features, level):
    print("\nLevel", level)
    possible_classes = list(set(Y))
    max_count = -np.inf
    max_class = possible_classes[0]
    for y in possible_classes:
        count_y = (Y == y).sum()
        print("Count of", y, "=", count_y)
        if count_y > max_count :
            max_count = count_y
            max_class = y   
    current_entropy = entropy(Y, possible_classes)
    print("Current Entropy is =", current_entropy)
    
    if len(possible_classes) == 1 or len(features) == 0:
        print("Reached leaf Node")
        return max_class
    selected_feature = None
    max_info_gain = -np.inf
    for f in features:
        info_gain, split_info = info_gain_on_f(X, Y, f)
        #print("info_gain, split_info", info_gain, split_info)
        #print("gain ratio", info_gain/split_info)

        if info_gain > max_info_gain:
            selected_feature = f
            max_info_gain = info_gain
            
    print("Splitting on feature", selected_feature,"with gain ratio", max_info_gain)
    splits = split(X, Y, selected_feature)
    unique_splits = list(set(splits))
    
    selected_feature_index = np.argwhere(features == selected_feature)
    features =  np.delete(features, selected_feature_index)
    
    for s in unique_splits:
        indices = (splits == s)
        X_split_i = X[indices, :]
        Y_split_i = Y[indices]
        decision_tree(X_split_i, Y_split_i, features, level + 1)
        
features = np.arange(len(X[0]))

max_class = decision_tree(X, Y, features, 0)



Level 0
Count of 0 = 50
Count of 1 = 50
Count of 2 = 50
Current Entropy is = 1.09861228867
Splitting on feature 1 with gain ratio 0.917956579775

Level 1
Count of 0 = 8
Count of 1 = 42
Count of 2 = 33
Current Entropy is = 0.936883911053
Splitting on feature 0 with gain ratio 0.781378530215

Level 2
Count of 0 = 8
Count of 1 = 24
Count of 2 = 6
Current Entropy is = 0.909707710327
Splitting on feature 2 with gain ratio 0.34739956058

Level 3
Count of 0 = 8
Current Entropy is = 0.0
Reached leaf Node

Level 3
Count of 1 = 7
Current Entropy is = 0.0
Reached leaf Node

Level 3
Count of 1 = 17
Count of 2 = 6
Current Entropy is = 0.573964491393
Splitting on feature 3 with gain ratio 0.159736973997

Level 4
Count of 1 = 3
Current Entropy is = 0.0
Reached leaf Node

Level 4
Count of 1 = 14
Count of 2 = 1
Current Entropy is = 0.244930026795
Reached leaf Node

Level 4
Count of 2 = 5
Current Entropy is = 0.0
Reached leaf Node

Level 2
Count of 1 = 18
Count of 2 = 27
Current Entropy is = 0.67301166

In [7]:
def entropy(Y, possible_classes):
    entropy = 0
    total_count = len(Y)
    for y in possible_classes :
        count_y = (Y == y).sum()
        p_i = count_y/total_count
        entropy -=  p_i * np.log(p_i)
    return entropy

def split(X, Y, feature):
    splits = np.zeros(len(Y))
    possible_values_current_feature = list(set( X[:, feature] ))

    group_label = 0
    for value in possible_values_current_feature :
        splits[ X[:, feature] == value ] = group_label
        group_label += 1
    return splits

def info_gain_on_f(X, Y, feature):
    info_gain = 0
    split_info = 0
    mod_Y = len(Y)
    
    splits = split(X, Y, feature)
    
    #print(splits)
    
    unique_splits = list(set(splits))
    
    for s in unique_splits:
        
        indices = (splits == s)
        
        X_split_i = X[indices, :]
        Y_split_i = Y[indices]
        
        #print([X_split_i, Y_split_i])
        
        mod_Y_i = len(Y_split_i)

        ratio_mod_Y_i_to_mod_Y = (mod_Y_i/mod_Y)
        possible_classes_Y_i = list(set(Y_split_i))
        info_gain += ((ratio_mod_Y_i_to_mod_Y) * entropy( Y_split_i, possible_classes_Y_i))
        split_info -= (ratio_mod_Y_i_to_mod_Y * np.log(ratio_mod_Y_i_to_mod_Y))
        
    return info_gain, split_info
    
def decision_tree(X, Y, features, level = 0):
    print("Level",  + 1l
    possible_classes = list(set(Y))
    max_count = -np.inf
    max_class = possible_classes[0]
    for y in possible_classes:
        count_y = (Y == y).sum()
        print("Count of", y, "=", count_y)
        if count_y > max_count :
            max_count = count_y
            max_class = y   
    current_entropy = entropy(Y, possible_classes)
    print("Current Entropy is =", current_entropy)
    
    if len(possible_classes) == 1 or len(features) == 0:
        print("Reached leaf Node")
        return max_class
    selected_feature = None
    max_info_gain = -np.inf
    for f in features:
        info_gain, split_info = info_gain_on_f(X, Y, f)
        print("info_gain, split_info", info_gain, split_info)
        print("gain ratio", info_gain/split_info)
        
        if info_gain > info_gain:
            selected_feature = f
            max_info_gain = info_gain
    print("Splitting on feature", selected_feature,"with gain ratio", max_info_gain)
    splits = split(X, Y, selected_feature)
    unique_splits = list(set(splits))
    del features[selected_feature]
    for s in unique_splits:
        indices = (splits == s)
        X_split_i = X[indices, :]
        Y_split_i = Y[indices]
        decision_tree(X_split_i, Y_split_i, features, level + 1)
        
features = np.arange(len(X[0]))
max_class = decision_tree(X_train, Y_train, features)

SyntaxError: invalid syntax (<ipython-input-7-66ce6d50f421>, line 46)

In [ ]:
def entropy(Y, possible_classes):
    entropy = 0
    total_count = len(Y)
    for y in possible_classes :
        count_y = (Y == y).sum()
        p_i = count_y/total_count
        entropy -=  p_i * np.log(p_i)
    return entropy

def split(X, Y, feature):
    splits = np.zeros(len(Y))
    possible_values_current_feature = list(set( X[:, feature] ))

    group_label = 0
    for value in possible_values_current_feature :
        splits[ X[:, feature] == value ] = group_label
        group_label += 1
    return splits

def info_gain_on_f(X, Y, feature):
    info_gain = 0
    split_info = 0
    mod_Y = len(Y)
    
    splits = split(X, Y, feature)
    
    print(splits)
    
    unique_splits = list(set(splits))
    
    for s in unique_splits:
        
        indices = (splits == s)
        
        X_split_i = X[indices, :]
        Y_split_i = Y[indices]
        
        #print([X_split_i, Y_split_i])
        
        mod_Y_i = len(Y_split_i)

        ratio_mod_Y_i_to_mod_Y = (mod_Y_i/mod_Y)
        possible_classes_Y_i = list(set(Y_split_i))
        info_gain += ((ratio_mod_Y_i_to_mod_Y) * entropy( Y_split_i, possible_classes_Y_i))
        split_info -= (ratio_mod_Y_i_to_mod_Y * np.log(ratio_mod_Y_i_to_mod_Y))
        
    return info_gain, split_info
    
def decision_tree(X, Y, features, level = 0):
    print("Level",  + 1l
    possible_classes = list(set(Y))
    max_count = -np.inf
    max_class = possible_classes[0]
    for y in possible_classes:
        count_y = (Y == y).sum()
        print("Count of", y, "=", count_y)
        if count_y > max_count :
            max_count = count_y
            max_class = y   
    current_entropy = entropy(Y, possible_classes)
    print("Current Entropy is =", current_entropy)
    
    if len(possible_classes) == 1 or len(features) == 0:
        print("Reached leaf Node")
        return max_class
    selected_feature = None
    max_info_gain = -np.inf
    for f in features:
        info_gain, split_info = info_gain_on_f(X, Y, f)
        print("info_gain, split_info", info_gain, split_info)
        print("gain ratio", info_gain/split_info)
        
        if info_gain > info_gain:
            selected_feature = f
            max_info_gain = info_gain
    print("Splitting on feature", selected_feature,"with gain ratio", max_info_gain)
    splits = split(X, Y, selected_feature)
    unique_splits = list(set(splits))
    for s in unique_splits:
        indices = (splits == s)
        X_split_i = X[indices, :]
        Y_split_i = Y[indices]
        decision_tree(X, Y, features, level + 1)
        
features = np.arange(len(X[0]))
max_class = decision_tree(X_train, Y_train, features)